<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="Logo HEIG-VD" style="width: 80px;" align="right"/>

# Cours APN - Labo 7 : Classification non supervisée

## Résumé
Le but de ce laboratoire est de réaliser une expérience de classification non-supervisée d'articles.  L'approche se base sur des vecteurs en basse dimension (*embeddings*) qui représentent les classes, qui seront comparés avec les vecteurs en basse dimension (*embeddings*) représentant les documents.  Ces *embeddings* seront obtenus soit au modèle `word2vec`.  La méthode sera testée sur un corpus d'articles provenant de rubriques connues, ce qui permettra d'évaluer les méthodes.

In [2]:
# Librairies générales
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

In [3]:
# Similarité entre vecteurs (mots, catégories ou textes)
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# Librairies pour l'évaluation
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

## 1. Préparation des données

Vous utiliserez un corpus d'environ 200'000 articles (titres et résumés) [diponibles sur Kaggle](https://www.kaggle.com/datasets/rmisra/news-category-dataset/versions/2) (V2, 80 Mo, nécessite un login) mais dont une copie est **fournie sur Cyberlearn**.  Dans cette partie, vous allez:
  - a. charger le corpus fourni au format JSON, l'explorer et afficher des statistiques
  - b. définir une fonction de normalisation de textes (avec la librairie `utils.py` déjà utilisée)
  - c. définir une fonction d'extraction des textes avec leur catégorie
***

**a.** Veuillez charger le corpus à l'aide des instructions données ci-dessous, puis afficher les statistiques suivantes :
  * un exemple d'article
  * nombre total d'articles
  * nombre d'articles pour chaque catégorie (ou classe) par ordre décroissant
  * nombre d'articles sans `headline`
  * nombre d'articles sans `short_description`
  * nombre d'articles dont la longueur de `headline + short_description` est inférieure ou égale à 2 caractères
  * longueur moyenne de `headline + short_description`

In [5]:
import json
from collections import Counter # pour calculer facilement le nombre d'articles par categorie

In [6]:
corpus  = []
with open('News_Category_Dataset_v2.json', mode='r', errors='ignore') as json_file:
    for dic in json_file:
        corpus.append(json.loads(dic))

In [7]:
# Statistiques générales
total_articles = len(corpus)
categories_count = {}
missing_headline_count = 0
missing_description_count = 0
short_articles_count = 0
total_lengths = []

for article in corpus:
    # Compter les catégories
    category = article.get('category', 'Unknown')
    categories_count[category] = categories_count.get(category, 0) + 1

    # Vérifier les articles sans headline ou description
    headline = article.get('headline', '')
    short_description = article.get('short_description', '')
    if not headline:
        missing_headline_count += 1
    if not short_description:
        missing_description_count += 1

    # Vérifier la longueur combinée de headline et short_description
    combined_length = len(headline) + len(short_description)
    if combined_length <= 2:
        short_articles_count += 1

    # Ajouter à la liste des longueurs
    total_lengths.append(combined_length)

# Calculer la longueur moyenne
average_length = np.mean(total_lengths) if total_lengths else 0

# Trier les catégories par ordre décroissant
sorted_categories = sorted(categories_count.items(), key=lambda x: x[1], reverse=True)

# Afficher les résultats
print("Exemple d'article :", corpus[0])
print("Nombre total d'articles :", total_articles)
print("Nombre d'articles par catégorie :", sorted_categories)
print("Nombre d'articles sans 'headline' :", missing_headline_count)
print("Nombre d'articles sans 'short_description' :", missing_description_count)
print("Nombre d'articles avec longueur 'headline + short_description' ≤ 2 :", short_articles_count)
print("Longueur moyenne de 'headline + short_description' :", average_length)

Exemple d'article : {'category': 'CRIME', 'headline': 'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV', 'authors': 'Melissa Jeltsen', 'link': 'https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89', 'short_description': 'She left her husband. He killed their children. Just another day in America.', 'date': '2018-05-26'}
Nombre total d'articles : 200853
Nombre d'articles par catégorie : [('POLITICS', 32739), ('WELLNESS', 17827), ('ENTERTAINMENT', 16058), ('TRAVEL', 9887), ('STYLE & BEAUTY', 9649), ('PARENTING', 8677), ('HEALTHY LIVING', 6694), ('QUEER VOICES', 6314), ('FOOD & DRINK', 6226), ('BUSINESS', 5937), ('COMEDY', 5175), ('SPORTS', 4884), ('BLACK VOICES', 4528), ('HOME & LIVING', 4195), ('PARENTS', 3955), ('THE WORLDPOST', 3664), ('WEDDINGS', 3651), ('WOMEN', 3490), ('IMPACT', 3459), ('DIVORCE', 3426), ('CRIME', 3405), ('MEDIA', 2815), ('WEIRD NEWS', 2670), ('GREEN', 2622), ('WORLDPOST', 2579), ('RELIGION', 2556), ('

**b.** Veuillez définir une fonction de nettoyage et normalisation des textes, qui vise à réduire la diversité du vocabulaire (lemmatisation, suppression des ponctuations, nombres, ou *stopwords*, etc.).  Veuillez utiliser les fonctions fournies dans la librairie `utils.py` fournie sur Cyberlearn et déjà vue au labo 3 (groupement hiérarchique de films).  Votre fonction devra prendre en entrée un texte non-tokenisé (une chaîne de caractères) et retournera une chaîne de caractères également, mais avec tous les tokens retenus.

In [8]:
from utils import *

def normalize_text(text):
    return lemmatize_text(remove_stopwords(remove_punctuation(text)))

**c.** Veuillez écrire une fonction qui sélectionne les articles d'une ou plusieurs catégories (données sous forme de liste, p.ex. `['MONEY', 'SCIENCE']`) et retourne leurs textes et leurs catégories.  Plus précisément :
* la fonction retourne une liste de textes et une liste de catégories de même longueur (au texte *i* correspond la catégorie ou classe *i*)
* le texte de chaque article est composé de son `headline` et de sa `short_description`, séparés par un point+espace
* si `normalize=True`, la fonction normalise les textes (note : ça ne sera pas toujours souhaitable dans les expériences suivantes)
* on ne retient dans le résultat que les textes dont la longueur finale est supérieure à 3 caractères.

In [9]:
def select_texts_categories(corpus, selected_categories, normalize=True):
    texts = list()
    categories = list()

    for article in corpus:
        category = article.get('category', 'Unknown')
        if category in selected_categories:

            headline = article.get('headline')
            short_desc = article.get('short_description')

            text = headline + '. ' + short_desc
            if normalize:
                text = normalize_text(text)

            if len(text) > 3:
                texts.append(text)
                categories.append(category)

    return texts, categories

**d.** Veuillez exécuter la fonction, en appliquant la normalisation des textes, puis afficher un exemple de résultat et commenter brièvement son contenu.  Le code est donné ci-dessous, et vous devez ajouter votre commentaire.

In [10]:
selected_categories = ['MONEY', 'SCIENCE']
texts, categories = select_texts_categories(corpus, selected_categories, normalize=True)
print("Exemple :", categories[142], ':', texts[142])

Exemple : SCIENCE : fireball rip through siberian sky in brilliant display of light frankly i scar i think bomb witness say


*Commentaire :* On peut voir que la catégorie correspond au texte et que la normalisation fonctionne (pas de ponctuation, tout en minuscule, etc...). Si on désactive la normalisation, on peut également clairement voir la différence entre le titre de l'article et la courte description.

___
## 2. Classification non supervisée avec des embeddings word2vec

La méthode de classification proposée comporte trois étapes.  Le but de cette partie est de définir une fonction pour chacune d'elles.  Au début, une (re)prise en main de word2vec est demandée.
* a. prise en main de word2vec
* b. création des représentations vectorielles (*embeddings*) des classes (catégories)
* c. création des représentations vectorielles (*embeddings*) d'un texte
* d. classification : comparer les similarités du vecteur de texte avec les vecteurs des classes, choisir la plus similaire
___

**a.** Prise en main de word2vec.  Vous avez déjà utilisé word2vec au Labo 4 sur la visualisation de vecteurs de mots (et Cémantix) mais ici vous utiliserez un modèle pour l'anglais.  Vous pouvez consulter la [documentation de gensim sur KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html#what-can-i-do-with-word-vectors)). 

In [11]:
from gensim.models import KeyedVectors
import gensim.downloader as api

In [25]:
# Si le modèle n'est pas téléchargé :
#w2v_model = api.downloader.load('word2vec-google-news-300')
# S'il l'est déjà, indiquer son emplacement :
path_to_model = "C:\\Users\\lcsch\\Downloads\\GoogleNews-vectors-negative300.bin"
w2v_model = KeyedVectors.load_word2vec_format(path_to_model, binary=True, unicode_errors='ignore')
# Attention, ce modèle prend environ 5 Go en mémoire. 

Veuillez afficher les mots les plus similaires selon ce modèle word2vec du mot 'science'.  Même question pour le mot 'money' (ce sont des noms de catégories).  Que pensez-vous du résultat ?  

In [26]:
print(w2v_model.most_similar('science'))
print(w2v_model.most_similar('money'))

[('faith_Jezierski', 0.6965422034263611), ('sciences', 0.6821076273918152), ('biology', 0.6775783896446228), ('scientific', 0.6535001993179321), ('mathematics', 0.6300910115242004), ('Hilal_Khashan_professor', 0.6153368353843689), ('impeach_USADA', 0.6149060130119324), ('professor_Kent_Redfield', 0.6144178509712219), ('physics_astronomy', 0.6105074882507324), ('bionic_prosthetic_fingers', 0.6083078980445862)]
[('monies', 0.7165061831474304), ('funds', 0.7055202722549438), ('moneys', 0.6289054751396179), ('dollars', 0.628852367401123), ('cash', 0.6151221394538879), ('vast_sums', 0.6057385802268982), ('fund', 0.5789709091186523), ('Money', 0.5733489394187927), ('taxpayer_dollars', 0.5693671107292175), ('Monies', 0.5586517453193665)]


Basé sur les résultats précédents et vos intuitions sur les articles qu'on peut trouver dans les catégories `['SCIENCE', 'MONEY']`, veuillez indiquer ici 5 à 10 mots représentatifs de chacune de ces deux catégories.

*Réponse :* 

- Pour 'science' : sciences, biologie, scientifique, maths, professor, physique, astronomie
- Pour 'money' : funds, moneys, dollar, cash, sum, tax


**b.** Création des représentations vectorielles des classes (catégories).

Veuillez définir une fonction qui retourne un vecteur (*embedding*) word2vec pour chacune des classes d'un tableau qui est fourni en argument de la fonction.  Consignes :

* la fonction retourne la moyenne des *embeddings* des mots-clés associés à chaque classe;
* pour les classes 'SCIENCE' et 'MONEY', elle prend les mots-clés que vous avez choisis ci-dessus (au (a)) ;
* pour les autres classes, elle demande à word2vec les `topn` voisins du nom de la classe (en minuscules) ;
  - si `topn = 0`, on utilise seulement le nom de la classe (en minuscules) ;
  - on suppose que le nom de la classe est connu de word2vec.


In [44]:
def cat_embedding_w2v(model, category_names, topn=30):
    category_embeddings = list()

    #print(model.most_similar('SCIENCE'))


    for cat in category_names:
        cat = cat.lower()

        neighbors = list()
        if cat == 'science':
            neighbors = ['sciences', 'biology', 'scientific', 'maths', 'professor', 'physics', 'astronomy']
        elif cat == 'money':
            neighbors = ['funds', 'moneys', 'dollar', 'cash', 'sum', 'tax']
        else:
            if topn == 0:
                neighbors = [cat]
            else:
                neighbors = [word for word, similarity in model.most_similar(cat, topn=topn)]

        vectors = [model.get_vector(word) for word in neighbors]
        category_embeddings.append(np.mean(vectors, axis=0))

    return category_embeddings

In [46]:

# Test : affiche-t-on 'True' ?
e1 = cat_embedding_w2v(w2v_model, ['SCIENCE'], topn=30)
e2 = cat_embedding_w2v(w2v_model, ['TECH'],    topn=30)
e3 = cat_embedding_w2v(w2v_model, ['TASTE'],   topn=30)
cosine_similarity(e1, e2) > cosine_similarity(e2, e3)

array([[ True]])

**c.** Création de la représentation vectorielle d'un texte

Veuillez définir une fonction qui prend un texte (*string*) en argument et retourne un vecteur (*embedding*) qui représente le texte.  Le texte doit être découpé en mots (tokenisé), puis on doit tester si chaque mot est connu du modèle word2vec, et si oui, on récupère le *embedding* du mot.  La fonction retourne la moyenne des *embeddings*, sauf si aucun mot du texte n'est connu du modèle word2vec, auquel cas elle retourne `[]`.

In [ ]:
from nltk import word_tokenize

In [ ]:
def text_embedding_w2v(model, text):
    embeddings = []
    tokens = nltk.word_tokenize(text)
    for word in tokens:
        if word not in model.vocab:
            continue
        embeddings.append(model.get_vector(word))

    if len(embeddings) == 0:
        return []
    else:
        return np.mean(embeddings, axis=0)
    
# todo : tester si ça marche


**d.** Classification non supervisée d'articles avec word2vec

Veuillez définir une fonction qui prend en entrée :
* un modèle word2vec
* une liste de textes à classifier
* une liste de catégories définies par leur nom en majuscules (p.ex. `['SCIENCE', 'MONEY']`)
et qui retourne le tableau avec la catégorie prédite pour chaque texte parmi les catégories données.  

Pour prédire la catégorie, la fonction calcule la similarité cosinus du *embedding* du texte avec chacun des *embeddings* des catégories, et choisit la catégorie qui a la plus grande similarité.  Si le texte n'a pas de *embedding* (parce qu'aucun de ses mots n'est connu du modèle), ou si plusieurs catégories ont la même similarité, on tire au sort la catégorie.

In [ ]:
def classify_w2v(model, texts, selected_categories, topn = 30):

    return cat_pred

**e.** Veuillez réaliser la classification non supervisée des articles des catégories `['SCIENCE', 'MONEY']`.  Afficher les scores obtenus et la matrice de confusion en utilisant les fonctions de `sklearn.metrics` importées au début du notebook.  Veuillez faire plusieurs essais pour optimiser les fonctions et leurs appels, et à la fin laisser votre meilleur résultat dans ce notebook.  

In [ ]:
selected_categories = ['SCIENCE', 'MONEY']


**f.** Veuillez réaliser une expérience de classification non supervisée sur les articles des catégories `['TECH', 'ARTS', 'COLLEGE']` en variant le paramètre `topn` de `classify_w2v` et en indiquant la meilleure valeur trouvée.

In [ ]:
selected_categories = ['TECH', 'ARTS', 'COLLEGE']


**g.** Veuillez comparer les deux expériences (points (e) et (f)) en termes de scores, de valeurs optimales de `topn`, et de l'impact du nettoyage de textes.   Veuillez donner votre opinion sur la qualité de la classification non supervisée avec word2vec.

**Fin du Labo.**  Veuillez nettoyer ce notebook en gardant seulement les résultats et les commentaires demandés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.